## Analyzing and fitting interferometric visibility datasets of circumstellar rings

Bringing your data from archive to paper-ready plots requires several steps, which we will follow and execute in this Notebook. Built in Python 3 and with CASA 5.4.0, but may (should) handle other versions.

## Step 0: Create list with all requested inputs

Here you need to decide if you are doing imaging and/or fitting and/or 
input some parameters that are needed for the imaging and/or fitting and/or postprocessing.


In [19]:
import numpy as np
import os

#Decide which you want to do, imaging and/or fitting and/or postprocessing
imaging=True
fit=False
postproc=False

casapath='/d1/boudica1/casa-release-5.4.0-68/bin'
sourcetag='GJ14'
workingdir='/d1/boudica1/lmatra'
vis=['GJ14_calibratedvis_cont.ms']
nvis=len(vis)
if not os.path.exists(workingdir+'/'+sourcetag):
    os.system('mkdir '+workingdir+'/'+sourcetag)
    print('Creating directory structure for object '+sourcetag+' at '+workingdir)
else:
    print('Directory structure for object '+sourcetag+' already exists at '+workingdir)

if imaging:
    #Imaging parameters
    mosaic=False
    if mosaic:
        phasecenter='J2000 22h57m39.449700 -29d37m22.68681'
    else:
        phasecenter=''
    weighting='natural'
    if weighting=='briggs':
        robust=0.5
    else:
        robust=''
    uvtaper=''
    interactive=True
    #Save parameters
    np.save(workingdir+'/'+sourcetag+'/imagepars.npy', [casapath,sourcetag,workingdir,vis,nvis,mosaic,phasecenter,weighting,robust,uvtaper,interactive])

if fit:
    #Parameters for fit
    Lstar=0.111  #Solar luminosities
    dist=14.7  #pc
    imsize=24.0     #arcsec, used for radiative transfer. >>2x belt outer radius, but not too large or it will slow down computation.
    #imsize above is also the size of the grid over which the model is setup in RADMC. No disk will be put beyond this.

    #Add disk parameters
    fluxdensity=1.8e-3 #Jy
    rmid=99.0/dist # Radial peak location of Gaussian surface density, arcsec
    sigma=32.0/2.35/dist #standard deviation of radial Gaussian surface density
    useh=True
    if useh:
        h=0.05 #aspect ratio of belt, constant with radius, whose vertical density structure at radius r is a Gaussian with width hr.
    else:
        h=0.03
    incl=65.0 #inclination, degrees from face-on
    posang=5.5 #position angle, East of North.

    #Add star if wanted/needed
    star=True
    if star:
        fstar=4e-5 #Jy
    
    #Add extra parameters independently for each of the visibility datasets.
    if nvis>=1:
        dRA=[-0.07,0.1,0.1] #RA offset of star+disk from phase center of observations
        dDec=[0.14,-0.1,-0.1] #Dec offset of star+disk from phase center of observations
        wtfact=[0.289,0.289,1e-4] #factors by which incorrect weights should be multiplied by
    else:
        dRA=[-0.07] #RA offset of star+disk from phase center of observations
        dDec=[0.14] #Dec offset of star+disk from phase center of observations
        wtfact=[0.289] #factors by which incorrect weights should be multiplied by

    #Add galaxies if needed
    ngal=0
    if ngal>=1:
        resolved=[True, False, False] # if definitely resolved, use 2D Gaussian as galaxy model (6 free parameters), and set resolved=True for that galaxy. 
    #Otherwise, use point source (3 free parameters) by setting resolved=False.
        fbkg=[285e-6,500e-6,500e-6] #Flux (Jy)
        dRAbkg=[2.97,2.97,2.97] # RA offset (")
        dDecbkg=[1.81,1.81,1.81] # Dec offset (")
        sigmagal=[0.3,0,0] # sigma (")
        PAgal=[28.0,0,0] # PA (deg)
        incgal=[49.0,0,0] # inc (deg)
    #Save parameters
    np.save(workingdir+'/'+sourcetag+'/fitpars.npy', [Lstar,dist,imsize, fluxdensity,rmid,sigma,useh,h,incl,posang,star,fstar,nvis,dRA,dDec,wtfact
             ,ngal,resolved,fbkg,dRAbkg,dDecbkg,sigmagal,PAgal,incgal])


Directory structure for object GJ14 already exists at /d1/boudica1/lmatra
